In [1]:
#remove irrelevant entries if the formulation was unspecified

def remove_irrelevant(input_array, phos_input, ion_input, peg_input, dna_input): 
    query_array = input_array

    #remove rows if the formulation doesn't use that component
    if phospholipid != 0:
        query_array = input_array[input_array[:,col_phos] == phospholipid]

    if ionizable_lipid != 0:
        query_array = query_array[query_array[:,col_ion] == ionizable_lipid]

    if lipid_PEG != 0:
        query_array = query_array[query_array[:,col_peg] == lipid_PEG]

    if surface_DNA != 0:
        query_array = query_array[query_array[:,col_dna] == surface_DNA]

    #Give a heads up
    if np.prod(query_array.shape) == 0:
        print("CAUTION: No training data for the combination of inputted reagents\n")
    
    return(query_array)
 


In [2]:
import sys
import numpy as np
import pandas as pd
from io import *
from sklearn import linear_model


#init lib path
allforms_path = "allforms_data_clean.csv"

#########################################################
###     init column vars      ###
#identities
col_lib = 0
col_phos = 1
col_ion = 2
col_peg = 3
col_dna = 4

#formulation (percent comp)
col_per_phos = 5
col_per_chol = 6
col_per_ion = 7
col_per_peg = 8

#results
col_encap = 9
col_size = 10 #in nm
col_stdev = 11 #nm

###########################################################

#init input vars. Users will be prompted to enter a number corresponded to the reagent provided in the prompt
phospholipid = 1
ionizable_lipid = 2
lipid_PEG = 2
surface_DNA = 0
user_size = 0

###########################################################

#convert raw lib into numpy array
allforms = np.genfromtxt(allforms_path, delimiter=",", skip_header=1)

#isolate relevant data
query_array = remove_irrelevant(allforms, phospholipid, ionizable_lipid, lipid_PEG, surface_DNA)





In [13]:

#define input/output arrays and build model

focus_percentage_training = query_array[:,(col_dna+1):(col_encap)] #molar percentages of relevant formulations
all_percentage_training = allforms[:,(col_dna+1):(col_encap)] #molar percentages of all formulations

focus_size_training = query_array[:,(col_size):(col_size+1)] #molar percentages of relevant formulations
all_size_training = allforms[:,(col_size):(col_size+1)] #molar percentages of all formulations


#focused_model = linear_model()

[[1.080e+02 1.000e+00 2.000e+00 2.000e+00 1.000e+00 2.250e+01 2.500e+01
  5.000e+01 2.500e+00 8.300e+01 1.900e+02 9.100e+01]
 [1.012e+03 1.000e+00 2.000e+00 2.000e+00 1.000e+00 1.250e+01 3.500e+01
  5.000e+01 2.500e+00 8.700e+01 1.780e+02 8.300e+01]
 [1.013e+03 1.000e+00 2.000e+00 2.000e+00 1.000e+00 2.000e+01 2.500e+01
  5.000e+01 5.000e+00 8.500e+01 2.310e+02 8.100e+01]
 [1.015e+03 1.000e+00 2.000e+00 2.000e+00 1.000e+00 1.000e+01 3.500e+01
  5.000e+01 5.000e+00 8.300e+01 1.870e+02 8.400e+01]]
[[22.5 25.  50.   2.5]
 [12.5 35.  50.   2.5]
 [20.  25.  50.   5. ]
 [10.  35.  50.   5. ]]
